In [1]:
# --------------------------------------------
#                   TEST LCU
# --------------------------------------------
from src import lcu
lcu = lcu.LCU()

instr_list = ["NOP", "EXIT", "SADD R1, ZERO, LAST", "SADD R1, SRF(3), LAST", "SADD R1, 7, ONE", "SSUB SRF(4), SRF(4), SRF(4)", "JUMP 7, ONE", "BGEPD ZERO, ONE, 5"]

for instr in instr_list:
    srf_read, srf_str, word = lcu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

NOP: -1, -1, 0x0
EXIT: -1, -1, 0x1c00
SADD R1, ZERO, LAST: -1, -1, 0xd4340
SADD R1, SRF(3), LAST: 3, -1, 0x94340
SADD R1, 7, ONE: -1, -1, 0xfc347
SSUB SRF(4), SRF(4), SRF(4): 4, 4, 0x90400
JUMP 7, ONE: -1, -1, 0xfda07
BGEPD ZERO, ONE, 5: -1, -1, 0xdd605


In [2]:
# --------------------------------------------
#                   TEST LSU
# --------------------------------------------
from src import lsu
lsu = lsu.LSU()

instr_list = ["SADD R0, ONE, ONE/LD.VWR VWR_A", "SADD R0, SRF(5), ONE/SH.IL.UP", "SADD SRF(5), SRF(5), ONE/LD.VWR SRF"]

for instr in instr_list:
    srf_read, srf_str, word = lsu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

SADD R0, ONE, ONE/LD.VWR VWR_A: -1, -1, 0x45538
SADD R0, SRF(5), ONE/SH.IL.UP: 5, -1, 0xc4538
SADD SRF(5), SRF(5), ONE/LD.VWR SRF: 5, 5, 0x5c530


In [3]:
# --------------------------------------------
#                   TEST RCs
# --------------------------------------------
from src import rc
rc = rc.RC()

instr_list = ["NOP", "SADD VWR_A, VWR_A, VWR_B", "SADD VWR_A, SRF(3), VWR_B", "LOR R0, RCB, MIN_INT", "MUL.FP R0, RCB, MIN_INT"]
# DIV.FP R0, RCB, MIN_INT") Error: float point div not supported
for instr in instr_list:
    srf_read, srf_str, vwr_str, word = rc.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

NOP: -1, -1, 0x0
SADD VWR_A, VWR_A, VWR_B: -1, -1, 0x420
SADD VWR_A, SRF(3), VWR_B: 3, -1, 0xc420
LOR R0, RCB, MIN_INT: -1, -1, 0x1f522
MUL.FP R0, RCB, MIN_INT: -1, -1, 0x1f5a2


In [4]:
# --------------------------------------------
#                   TEST MXCU
# --------------------------------------------
from src import mxcu
mxcu = mxcu.MXCU()

instr_list = ["NOP", "SADD R1, ONE, LAST", "LOR R1, ONE, SRF(3)"]
srf_sel = -1
srf_we = 0
alu_srf_write = 0
vwr_row_we = [0,0,0,0]
vwr_sel = 0
for instr in instr_list:
    word = mxcu.asmToHex(instr, srf_sel, srf_we, alu_srf_write, vwr_row_we, vwr_sel)
    print(instr + ": " + word.get_word_in_hex())

NOP: 0x0
SADD R1, ONE, LAST: 0x5699000
LOR R1, ONE, SRF(3): 0x54690c0


In [5]:
# --------------------------------------------
#                   COMPILE
# --------------------------------------------
# Config kernel
kernel_path='./kernels/test_asm/'
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 15 # Number of instructions per column
imem_add_start = 3 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version="_v1"
from src import simulator
sim = simulator.SIMULATOR()
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

ASM to Hex
Processing file: ./kernels/test_asm/instructions_asm_v1.csv...


In [6]:
kernel_path='./kernels/test_asm/'
kernel_number = 2 # Kernel number (from 1 to 15)
column_usage = [True, True] # Columns to use
nInstrPerCol = 5 # Number of instructions per column
imem_add_start = 30 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version="_v2"
#sim = simulator.SIMULATOR() # Coment this to store two kernels at the same time in the imem
# Using two cols
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

ASM to Hex
Processing file: ./kernels/test_asm/instructions_asm_v2.csv...


In [7]:
# --------------------------------------------
#                   SIMULATE
# --------------------------------------------
from src.simulator import *
from random import randint

sim = SIMULATOR()

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/test_asm/' # Path
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 15 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version="_v1"

sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)

# --------------------------------------------
#                LOAD SPM DATA
# --------------------------------------------
# Load vector by vector in the positions you want
random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
nline = 3
sim.setSPMLine(nline, random_vector)
# Or, load the entire content of the SPM
data = []
for nline in range(SPM_NLINES):
    random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
    data.append(random_vector)
sim.loadSPMData(data)

# --------------------------------------------
#              COMPILE ASM TO HEX
# --------------------------------------------
# If hexadecimal is not provided, here we generate it
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

# --------------------------------------------
#                 LOAD KERNEL
# --------------------------------------------

# This needs the hex instructions, if you don't provide them, generate then compiling the asm
sim.kernel_load(kernel_path, version=version + "_autogen", kernel_number=kernel_number)

# --------------------------------------------
#               SIMULATE EXECUTION
# --------------------------------------------
show_lcu = []
show_srf = []
show_lsu = []
show_rcs = [[],[],[],[]]
show_mxcu = []
display_ops = [show_lcu, show_lsu, show_mxcu, show_rcs, show_srf]

sim.displaySPMLine(3)

# Do not run since it is an infinite loop
#sim.run(kernel_number, display_ops=display_ops)

ASM to Hex
Processing file: ./kernels/test_asm/instructions_asm_v1.csv...
Processing file: ./kernels/test_asm/instructions_hex_v1_autogen.csv...
SPM 3: [91, 26, 61, 7, 8, 27, 100, 72, 87, 78, 37, 94, 2, 99, 1, 67, 13, 6, 63, 68, 6, 50, 6, 46, 93, 53, 69, 65, 86, 83, 64, 65, 81, 31, 100, 45, 61, 73, 76, 67, 60, 30, 92, 25, 77, 10, 14, 63, 54, 63, 9, 4, 44, 3, 56, 100, 96, 88, 5, 57, 18, 90, 28, 78, 3, 26, 2, 76, 65, 72, 72, 78, 34, 77, 12, 79, 6, 71, 3, 43, 12, 1, 23, 74, 38, 5, 60, 43, 8, 77, 61, 44, 69, 94, 53, 43, 20, 37, 23, 33, 95, 87, 57, 63, 35, 27, 32, 49, 79, 98, 98, 12, 100, 50, 41, 88, 74, 59, 2, 99, 63, 75, 26, 74, 45, 60, 9, 8, ]


In [8]:
kernel_path = './kernels/exit/' 
kernel_number = 1
column_usage = [True, False] # Columns to use
nInstrPerCol = 1 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileAsmToHex(kernel_path, kernel_number)

ASM to Hex
Processing file: ./kernels/exit/instructions_asm.csv...
Hex to ASM
Processing file: ./kernels/exit/instructions_hex_autogen.csv...


In [9]:
kernel_path = './kernels/add_vectors/' 
sim.compileHexToAsm(kernel_path)

Hex to ASM
Processing file: ./kernels/add_vectors/instructions_hex.csv...


In [10]:
# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/add_vectors/' 
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 37 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version=""
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileHexToAsm(kernel_path, version=version)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)


Hex to ASM
Processing file: ./kernels/add_vectors/instructions_hex.csv...
ASM to Hex
Processing file: ./kernels/add_vectors/instructions_asm.csv...


In [11]:

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/mf_q64_erosion/' 
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 44 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version=""
sim = simulator.SIMULATOR()
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileHexToAsm(kernel_path)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)


Hex to ASM
Processing file: ./kernels/mf_q64_erosion/instructions_hex.csv...
ASM to Hex
Processing file: ./kernels/mf_q64_erosion/instructions_asm.csv...


In [12]:
from src import *
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x1ce8e12

mxcu_imem = MXCU_IMEM()
mxcu_imem.set_word(imem_word, imem_pos)
asm = mxcu_imem.get_instr_pseudo_asm(imem_pos)
print("Pseudo-Assembly representation: " + asm)
mxcu_imem.get_instruction_info(imem_pos)
hex = mxcu_imem.get_word_in_hex(imem_pos)
print("Hex: " + hex)

print("ORIG -------------------------")


# Load an existing imem word and decode it
imem_pos = 2
imem_word = 0x1ce8ed2

mxcu_imem.set_word(imem_word, imem_pos)
asm = mxcu_imem.get_instr_pseudo_asm(imem_pos)
print("Pseudo-Assembly representation: " + asm)
mxcu_imem.get_instruction_info(imem_pos)

print("MXCU -------------------------")

# Load an existing imem word and decode it
imem_pos = 2
imem_word = 0x1ce8ed2

mxcu_imem = MXCU_IMEM()
mxcu_imem.set_word(imem_word, imem_pos)
asm = mxcu_imem.get_instr_pseudo_asm(imem_pos)
print("Pseudo-Assembly representation: " + asm)
mxcu_imem.get_instruction_info(imem_pos)



Pseudo-Assembly representation: LOR R0, R3, ZERO
Writing to VWR rows [1] of VWR_B
Writing from LSU ALU to SRF register 0
Performing ALU operation LOR between operands R3 and ZERO
Writing ALU result to MXCU register 0
Hex: 0x1ce8e12
ORIG -------------------------
Pseudo-Assembly representation: LOR R0, R3, ZERO
Writing to VWR rows [1] of VWR_B
Writing from LSU ALU to SRF register 3
Performing ALU operation LOR between operands R3 and ZERO
Writing ALU result to MXCU register 0
MXCU -------------------------
Pseudo-Assembly representation: LOR R0, R3, ZERO
Writing to VWR rows [1] of VWR_B
Writing from LSU ALU to SRF register 3
Performing ALU operation LOR between operands R3 and ZERO
Writing ALU result to MXCU register 0
